In [1]:
import pandas as pd
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', None) 
import numpy as np
import re
from collections import defaultdict

In [2]:
ids = pd.read_csv('Raw/player_ids.csv')
ids.head()

,gsis_id,stats_id,mapped_ids,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo
0,00-0016919,3727,3727,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2014,adam vinatieri,adam vinatieri_3727,vinatieri
1,00-0016919,3727,3727,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,348.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2015,adam vinatieri,adam vinatieri_3727,vinatieri
2,00-0016919,3727,3727,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,359.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2016,adam vinatieri,adam vinatieri_3727,vinatieri
3,00-0016919,3727,3727,11,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,371.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2017,adam vinatieri,adam vinatieri_3727,vinatieri
4,00-0016919,3727,3727,9,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,380.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2018,adam vinatieri,adam vinatieri_3727,vinatieri


In [3]:
ids[ids['cleaned_name_yahoo'] == 'travis hunter']

,gsis_id,stats_id,mapped_ids,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo
11772,00-0040718,41787,99001,8,WR,nfl.p.41787,Jax,Jacksonville Jaguars,nfl.t.30,https://sports.yahoo.com/nfl/teams/jacksonville/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/VMwr0OLB8DWK...,https://s.yimg.com/iu/api/res/1.2/VMwr0OLB8DWK...,False,False,False,0,Travis,Travis,Travis Hunter,Hunter,99001,461.p.99001,O,WR,https://sports.yahoo.com/nfl/players/41787,2025,travis hunter,travis hunter_99001,hunter


In [4]:
ids = ids.rename(columns={'stats_id':'true_y_id'})

In [5]:
ids.head()

,gsis_id,true_y_id,mapped_ids,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo
0,00-0016919,3727,3727,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2014,adam vinatieri,adam vinatieri_3727,vinatieri
1,00-0016919,3727,3727,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,348.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2015,adam vinatieri,adam vinatieri_3727,vinatieri
2,00-0016919,3727,3727,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,359.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2016,adam vinatieri,adam vinatieri_3727,vinatieri
3,00-0016919,3727,3727,11,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,371.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2017,adam vinatieri,adam vinatieri_3727,vinatieri
4,00-0016919,3727,3727,9,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,380.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2018,adam vinatieri,adam vinatieri_3727,vinatieri


In [6]:
ids.columns.duplicated().sum()

np.int64(0)

In [7]:
ids[ids['mapped_ids'] != ids['player_id']]

,gsis_id,true_y_id,mapped_ids,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo


In [8]:
ids = ids[['gsis_id', 'cleaned_name_yahoo', 'true_y_id', 'editorial_team_key', 'editorial_team_abbr', 'player_id', 'player_key', 'primary_position', 'season', 'last_name_yahoo']]

In [9]:
ids['gs_unq'] = ids['gsis_id'] + '_' + ids['season'].astype(str)

In [10]:
ids.head()

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq
0,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,331.p.3727,K,2014,vinatieri,00-0016919_2014
1,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,348.p.3727,K,2015,vinatieri,00-0016919_2015
2,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,359.p.3727,K,2016,vinatieri,00-0016919_2016
3,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,371.p.3727,K,2017,vinatieri,00-0016919_2017
4,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,380.p.3727,K,2018,vinatieri,00-0016919_2018


In [11]:
y_stats = pd.read_csv('Raw/stats_yahooleague_raw_14to25.csv')
y_stats.head()

,Unnamed: 0,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,player_points.coverage_type,player_points.season,player_points.total,player_stats.coverage_type,player_stats.season,player_stats.stats,position_type,primary_position,url
0,0,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,season,2014,153.00,season,2014,"[{'stat': {'stat_id': 19, 'value': 0.0}}, {'st...",K,K,https://sports.yahoo.com/nfl/players/3727
1,1,9,TE,nfl.p.3950,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,False,False,False,0,Tony,Tony,Tony Gonzalez,Gonzalez,3950,331.p.3950,season,2014,0.00,season,2014,"[{'stat': {'stat_id': 2, 'value': 0.0}}, {'sta...",O,TE,https://sports.yahoo.com/nfl/players/3950
2,2,4,QB,nfl.p.4256,Den,Denver Broncos,nfl.t.7,https://sports.yahoo.com/nfl/teams/denver/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,False,False,False,0,Peyton,Peyton,Peyton Manning,Manning,4256,331.p.4256,season,2014,398.23,season,2014,"[{'stat': {'stat_id': 2, 'value': 0.0}}, {'sta...",O,QB,https://sports.yahoo.com/nfl/players/4256
3,3,4,K,nfl.p.4269,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['K'],small,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,False,False,False,0,Phil,Phil,Phil Dawson,Dawson,4269,331.p.4269,season,2014,125.00,season,2014,"[{'stat': {'stat_id': 19, 'value': 0.0}}, {'st...",K,K,https://sports.yahoo.com/nfl/players/4269
4,4,10,QB,nfl.p.4416,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,False,False,False,0,Matt,Matt,Matt Hasselbeck,Hasselbeck,4416,331.p.4416,season,2014,20.79,season,2014,"[{'stat': {'stat_id': 2, 'value': 0.0}}, {'sta...",O,QB,https://sports.yahoo.com/nfl/players/4416


In [12]:
y_stats = y_stats.drop(columns='Unnamed: 0')
y_stats.head()

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,player_points.coverage_type,player_points.season,player_points.total,player_stats.coverage_type,player_stats.season,player_stats.stats,position_type,primary_position,url
0,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,season,2014,153.00,season,2014,"[{'stat': {'stat_id': 19, 'value': 0.0}}, {'st...",K,K,https://sports.yahoo.com/nfl/players/3727
1,9,TE,nfl.p.3950,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,False,False,False,0,Tony,Tony,Tony Gonzalez,Gonzalez,3950,331.p.3950,season,2014,0.00,season,2014,"[{'stat': {'stat_id': 2, 'value': 0.0}}, {'sta...",O,TE,https://sports.yahoo.com/nfl/players/3950
2,4,QB,nfl.p.4256,Den,Denver Broncos,nfl.t.7,https://sports.yahoo.com/nfl/teams/denver/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,False,False,False,0,Peyton,Peyton,Peyton Manning,Manning,4256,331.p.4256,season,2014,398.23,season,2014,"[{'stat': {'stat_id': 2, 'value': 0.0}}, {'sta...",O,QB,https://sports.yahoo.com/nfl/players/4256
3,4,K,nfl.p.4269,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['K'],small,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,False,False,False,0,Phil,Phil,Phil Dawson,Dawson,4269,331.p.4269,season,2014,125.00,season,2014,"[{'stat': {'stat_id': 19, 'value': 0.0}}, {'st...",K,K,https://sports.yahoo.com/nfl/players/4269
4,10,QB,nfl.p.4416,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,False,False,False,0,Matt,Matt,Matt Hasselbeck,Hasselbeck,4416,331.p.4416,season,2014,20.79,season,2014,"[{'stat': {'stat_id': 2, 'value': 0.0}}, {'sta...",O,QB,https://sports.yahoo.com/nfl/players/4416


In [13]:
y_stats = y_stats[['bye_weeks.week', 'player_key', 'player_points.total']]

In [14]:
y_stats['player_key'].nunique()

14096

In [15]:
ids['player_key'].nunique()

12157

In [16]:
ids['player_key'].isin(y_stats['player_key']).sum()

np.int64(12157)

In [17]:
y_stats[~(y_stats['player_key'].isin(ids['player_key']))]

,bye_weeks.week,player_key,player_points.total
107,9,331.p.8054,1.8
348,12,331.p.24746,0.0
385,11,331.p.24898,0.0
423,4,331.p.25050,0.0
432,5,331.p.25123,0.0
...,...,...,...
14057,11,461.p.42587,0.0
14058,8,461.p.42592,0.0
14059,14,461.p.42596,0.0
14060,11,461.p.42597,0.0


In [18]:
ids_w_points = ids.merge(y_stats, how='inner', on='player_key')
ids_w_points

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total
0,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,331.p.3727,K,2014,vinatieri,00-0016919_2014,10,153.00
1,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,348.p.3727,K,2015,vinatieri,00-0016919_2015,10,119.00
2,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,359.p.3727,K,2016,vinatieri,00-0016919_2016,10,149.00
3,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,371.p.3727,K,2017,vinatieri,00-0016919_2017,11,123.00
4,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,380.p.3727,K,2018,vinatieri,00-0016919_2018,9,123.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12152,Houston Texans,houston texans,100034,nfl.t.34,Hou,100034,406.p.100034,DEF,2021,texans,Houston Texans_2021,10,156.38
12153,Houston Texans,houston texans,100034,nfl.t.34,Hou,100034,414.p.100034,DEF,2022,texans,Houston Texans_2022,6,171.25
12154,Houston Texans,houston texans,100034,nfl.t.34,Hou,100034,423.p.100034,DEF,2023,texans,Houston Texans_2023,7,205.00
12155,Houston Texans,houston texans,100034,nfl.t.34,Hou,100034,449.p.100034,DEF,2024,texans,Houston Texans_2024,14,242.00


In [19]:
ids_w_points.columns.duplicated().sum()

np.int64(0)

In [20]:
ids_w_points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12157 entries, 0 to 12156
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gsis_id              12157 non-null  object 
 1   cleaned_name_yahoo   12157 non-null  object 
 2   true_y_id            12157 non-null  int64  
 3   editorial_team_key   12157 non-null  object 
 4   editorial_team_abbr  12157 non-null  object 
 5   player_id            12157 non-null  int64  
 6   player_key           12157 non-null  object 
 7   primary_position     12157 non-null  object 
 8   season               12157 non-null  int64  
 9   last_name_yahoo      12157 non-null  object 
 10  gs_unq               12157 non-null  object 
 11  bye_weeks.week       12157 non-null  int64  
 12  player_points.total  12157 non-null  float64
dtypes: float64(1), int64(4), object(8)
memory usage: 1.2+ MB


In [21]:
nfl_player_stats = pd.read_csv('Raw/stats_player_nfl_raw_14to25.csv')
nfl_player_stats.head()

,Unnamed: 0,player_id,player_name,player_display_name,position,position_group,headshot_url,season,season_type,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,fantasy_points,fantasy_points_ppr
0,0,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,31,5,1,55.0,0.806452,0,6,8,5,6,0,0,0,1,1,2,1,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948,250,45,33,33,0,0,1.0,1,1,0,0,35,0.00,0.00
1,1,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,IND,4,30,44,301,2,0,2,-7,1,1,313,182,17,-0.628051,3.970076,0,0.961661,8,-11,0,0,0,0,-7.756981,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,16.94,16.94
2,2,00-0010346,P.Manning,Peyton Manning,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,DEN,16,395,597,4727,39,15,17,-118,2,1,5071,1882,225,121.816891,3.316656,2,0.932163,24,-24,0,3,1,2,-24.812411,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,2,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,1,3,0,0,0,3,20,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,312.68,312.68
3,3,00-0016919,A.Vinatieri,Adam Vinatieri,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,IND,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,31,1,0,53.0,0.967742,0,10,10,7,3,0,0,0,0,1,0,0,25;46;27;48;43;25;31;30;38;34;27;34;23;50;31;2...,46,NaN,1049,46,0,50,50,0,0,1.0,0,0,0,0,NaN,0.00,0.00
4,4,00-0018227,C.Woodson,Charles Woodson,FS,DB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,LV,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,75,8,30,4,21,0,1.0,10.0,1,4,35,8,0,0,0,0,0,0,1,1,0,0,0,3,28,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,0.00,0.00


In [22]:
nfl_player_stats = nfl_player_stats.drop(columns='Unnamed: 0')
nfl_player_stats.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,season,season_type,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,fantasy_points,fantasy_points_ppr
0,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,31,5,1,55.0,0.806452,0,6,8,5,6,0,0,0,1,1,2,1,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948,250,45,33,33,0,0,1.0,1,1,0,0,35,0.00,0.00
1,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,IND,4,30,44,301,2,0,2,-7,1,1,313,182,17,-0.628051,3.970076,0,0.961661,8,-11,0,0,0,0,-7.756981,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,16.94,16.94
2,00-0010346,P.Manning,Peyton Manning,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,DEN,16,395,597,4727,39,15,17,-118,2,1,5071,1882,225,121.816891,3.316656,2,0.932163,24,-24,0,3,1,2,-24.812411,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,2,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,1,3,0,0,0,3,20,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,312.68,312.68
3,00-0016919,A.Vinatieri,Adam Vinatieri,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,IND,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,31,1,0,53.0,0.967742,0,10,10,7,3,0,0,0,0,1,0,0,25;46;27;48;43;25;31;30;38;34;27;34;23;50;31;2...,46,NaN,1049,46,0,50,50,0,0,1.0,0,0,0,0,NaN,0.00,0.00
4,00-0018227,C.Woodson,Charles Woodson,FS,DB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,LV,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,75,8,30,4,21,0,1.0,10.0,1,4,35,8,0,0,0,0,0,0,1,1,0,0,0,3,28,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,0.00,0.00


In [23]:
nfl_player_stats['recent_team'].value_counts()

recent_team
DET    758
NYJ    755
WAS    754
NYG    754
HOU    743
TEN    739
SF     737
CLE    737
ARI    732
NO     729
CAR    725
BAL    725
IND    723
LAC    721
MIA    716
CHI    715
JAX    715
BUF    713
SEA    711
PHI    710
LV     709
NE     708
DAL    705
TB     702
DEN    700
ATL    689
CIN    687
LA     682
KC     680
PIT    673
GB     671
MIN    670
Name: count, dtype: int64

In [24]:
nfl_player_stats['recent_team'].nunique()

32

In [25]:
nfl_player_stats[nfl_player_stats['player_display_name'] == 'Phil Dawson']

,player_id,player_name,player_display_name,position,position_group,headshot_url,season,season_type,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,fantasy_points,fantasy_points_ppr
0,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,31,5,1,55.0,0.806452,0,6,8,5,6,0,0,0,1,1,2,1,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948,250,45,33,33,0,0,1.000000,1,1,0,0,35,0.0,0.0
1850,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2015,REG,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,27,1,2,54.0,0.888889,0,9,6,6,3,0,0,0,0,1,0,0,30;25;47;33;43;22;53;31;26;42;35;54;26;44;27;2...,45,28;41,848,45,69,20,21,0,1,0.952381,1,1,0,0,23,0.0,0.0
3696,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2016,REG,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18,21,3,0,53.0,0.857143,0,4,6,7,1,0,0,0,0,2,1,0,29;39;22;36;33;48;47;28;49;48;46;49;53;33;36;3...,53;44;48,NaN,702,145,0,33,34,1,0,0.970588,0,0,0,0,NaN,0.0,0.0
5552,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2017,REG,ARI,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32,40,6,2,57.0,0.800000,1,7,10,10,4,0,0,0,4,2,0,0,24;27;40;30;37;29;43;50;32;28;25;32;43;34;42;4...,32;42;36;32;38;40,51;45,1183,220,96,23,26,1,2,0.884615,1,1,0,0,30,0.0,0.0
7421,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2018,REG,ARI,10,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,8,3,0,43.0,0.625000,0,2,2,1,0,0,0,0,0,2,1,0,23;26;43;31;30,50;45;46,NaN,153,141,0,15,15,0,0,1.000000,0,0,0,0,NaN,0.0,0.0


In [26]:
nfl_player_stats['position'].value_counts()

position
WR     2685
LB     2125
CB     2026
DE     1795
RB     1785
DT     1570
TE     1501
OT     1259
G      1152
DB     1063
QB      904
OLB     721
SAF     617
C       581
K       495
S       468
P       435
FS      423
ILB     334
LS      332
MLB     218
FB      188
NT      154
DL       35
OL       11
Name: count, dtype: int64

In [27]:
nfl_player_stats = nfl_player_stats.loc[nfl_player_stats['position'].isin(['QB', 'WR', 'RB', 'TE', 'K'])]

In [28]:
nfl_player_stats['gs_unq'] = nfl_player_stats['player_id'] + '_' + nfl_player_stats['season'].astype(str)
nfl_player_stats.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,season,season_type,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,fantasy_points,fantasy_points_ppr,gs_unq
0,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,31,5,1,55.0,0.806452,0,6,8,5,6,0,0,0,1,1,2,1,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948,250,45,33,33,0,0,1.000000,1,1,0,0,35,0.00,0.00,00-0004091_2014
1,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,IND,4,30,44,301,2,0,2,-7,1,1,313,182,17,-0.628051,3.970076,0,0.961661,8,-11,0,0,0,0,-7.756981,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,16.94,16.94,00-0007091_2014
2,00-0010346,P.Manning,Peyton Manning,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,DEN,16,395,597,4727,39,15,17,-118,2,1,5071,1882,225,121.816891,3.316656,2,0.932163,24,-24,0,3,1,2,-24.812411,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,2,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,1,3,0,0,0,3,20,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,312.68,312.68,00-0010346_2014
3,00-0016919,A.Vinatieri,Adam Vinatieri,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,IND,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,31,1,0,53.0,0.967742,0,10,10,7,3,0,0,0,0,1,0,0,25;46;27;48;43;25;31;30;38;34;27;34;23;50;31;2...,46,NaN,1049,46,0,50,50,0,0,1.000000,0,0,0,0,NaN,0.00,0.00,00-0016919_2014
6,00-0019536,S.Graham,Shayne Graham,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,NO,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19,22,3,0,50.0,0.863636,0,6,9,3,1,0,0,0,0,2,1,0,31;50;27;30;30;29;44;27;48;36;31;29;37;40;31;2...,41;42;51,NaN,636,134,0,46,47,0,1,0.978723,0,0,0,0,NaN,0.00,0.00,00-0019536_2014


In [29]:
nfl_player_stats = nfl_player_stats.drop(columns=['player_id', 'player_name', 'player_display_name', 'position', 'position_group', 'headshot_url', 'season', 'season_type', 'fantasy_points', 'fantasy_points_ppr'])

In [30]:
nfl_player_stats.head()

,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,gs_unq
0,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,31,5,1,55.0,0.806452,0,6,8,5,6,0,0,0,1,1,2,1,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948,250,45,33,33,0,0,1.000000,1,1,0,0,35,00-0004091_2014
1,IND,4,30,44,301,2,0,2,-7,1,1,313,182,17,-0.628051,3.970076,0,0.961661,8,-11,0,0,0,0,-7.756981,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,00-0007091_2014
2,DEN,16,395,597,4727,39,15,17,-118,2,1,5071,1882,225,121.816891,3.316656,2,0.932163,24,-24,0,3,1,2,-24.812411,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,2,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,1,3,0,0,0,3,20,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,00-0010346_2014
3,IND,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,31,1,0,53.0,0.967742,0,10,10,7,3,0,0,0,0,1,0,0,25;46;27;48;43;25;31;30;38;34;27;34;23;50;31;2...,46,NaN,1049,46,0,50,50,0,0,1.000000,0,0,0,0,NaN,00-0016919_2014
6,NO,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19,22,3,0,50.0,0.863636,0,6,9,3,1,0,0,0,0,2,1,0,31;50;27;30;30;29;44;27;48;36;31;29;37;40;31;2...,41;42;51,NaN,636,134,0,46,47,0,1,0.978723,0,0,0,0,NaN,00-0019536_2014


In [31]:
nfl_player_stats[nfl_player_stats['gs_unq'].isin(ids_w_points['gs_unq'])]

,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,gs_unq
0,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,31,5,1,55.0,0.806452,0,6,8,5,6,0,0,0,1,1,2,1,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948,250,45,33,33,0,0,1.000000,1,1,0,0,35,00-0004091_2014
1,IND,4,30,44,301,2,0,2,-7,1,1,313,182,17,-0.628051,3.970076,0,0.961661,8,-11,0,0,0,0,-7.756981,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,00-0007091_2014
2,DEN,16,395,597,4727,39,15,17,-118,2,1,5071,1882,225,121.816891,3.316656,2,0.932163,24,-24,0,3,1,2,-24.812411,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.000000,0.000000,0,2,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,1,3,0,0,0,3,20,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,00-0010346_2014
3,IND,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,31,1,0,53.0,0.967742,0,10,10,7,3,0,0,0,0,1,0,0,25;46;27;48;43;25;31;30;38;34;27;34;23;50;31;2...,46,NaN,1049,46,0,50,50,0,0,1.000000,0,0,0,0,NaN,00-0016919_2014
6,NO,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19,22,3,0,50.0,0.863636,0,6,9,3,1,0,0,0,0,2,1,0,31;50;27;30;30;29;44;27;48;36;31;29;37;40;31;2...,41;42;51,NaN,636,134,0,46,47,0,1,0.978723,0,0,0,0,NaN,00-0019536_2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22871,LA,3,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,1,0,0,0,0,0,-1.549391,0,2,4,52,1,0,0,103,3,2,5.562502,0,0.504854,0.016807,0.048631,0.059252,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,00-0040737_2025
22873,SEA,7,0,0,0,0,0,0,0,0,0,0,0,

In [32]:
ids_w_points.loc[(~ids_w_points['gs_unq'].isin(nfl_player_stats['gs_unq']) & (ids_w_points['player_points.total'] <= 0))]

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total
6,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,399.p.3727,K,2020,vinatieri,00-0016919_2020,7,0.0
7,00-0006101,tony gonzalez,3950,nfl.t.1,Atl,3950,331.p.3950,TE,2014,gonzalez,00-0006101_2014,9,0.0
15,00-0004091,phil dawson,4269,nfl.t.22,Ari,4269,390.p.4269,K,2019,dawson,00-0004091_2019,12,0.0
18,00-0000108,david akers,4587,nfl.t.8,Det,4587,331.p.4587,K,2014,akers,00-0000108_2014,9,0.0
22,00-0019646,sebastian janikowski,5046,nfl.t.26,Sea,5046,371.p.5046,K,2017,janikowski,00-0019646_2017,6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11760,00-0040556,corey kiner,42386,nfl.t.16,Min,42386,461.p.42386,RB,2025,kiner,00-0040556_2025,6,0.0
11766,00-0040024,darius cooper,42464,nfl.t.21,Phi,42464,461.p.42464,WR,2025,cooper,00-0040024_2025,9,0.0
11767,00-0040315,jacorey brooks,42482,nfl.t.28,Was,42482,461.p.42482,WR,2025,brooks,00-0040315_2025,12,0.0
11768,00-0040617,luke floriea,42507,nfl.t.5,Cle,42507,461.p.42507,WR,2025,floriea,00-0040617_2025,9,0.0


In [33]:
nfl_player_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7370 entries, 0 to 22887
Columns: 104 entries, recent_team to gs_unq
dtypes: float64(14), int64(84), object(6)
memory usage: 5.9+ MB


In [34]:
ids_w_ply_stats = ids_w_points.merge(nfl_player_stats, how ='outer', on='gs_unq')
ids_w_ply_stats.head(10)

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list
0,00-0000108,david akers,4587.0,nfl.t.8,Det,4587.0,331.p.4587,K,2014.0,akers,00-0000108_2014,9.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,331.p.4269,K,2014.0,dawson,00-0004091_2014,4.0,125.00,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,31.0,5.0,1.0,55.0,0.806452,0.0,6.0,8.0,5.0,6.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948.0,250.0,45.0,33.0,33.0,0.0,0.0,1.000000,1.0,1.0,0.0,0.0,35
2,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,348.p.4269,K,2015.0,dawson,00-0004091_2015,9.0,102.00,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,27.0,1.0,2.0,54.0,0.888889,0.0,9.0,6.0,6.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30;25;47;33;43;22;53;31;26;42;35;54;26;44;27;2...,45,28;41,848.0,45.0,69.0,20.0,21.0,0.0,1.0,0.952381,1.0,1.0,0.0,0.0,23
3,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,359.p.4269,K,2016.0,dawson,00-0004091_2016,9.0,95.00,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,21.0,3.0,0.0,53.0,0.857143,0.0,4.0,6.0,7.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,29;39;22;36;33;48;47;28;49;48;46;49;53;33;36;3...,53;44;48,NaN,702.0,145.0,0.0,33.0,34.0,1.0,0.0,0.970588,0.0,0

In [35]:
ids_w_ply_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12191 entries, 0 to 12190
Columns: 116 entries, gsis_id to gwfg_distance_list
dtypes: float64(103), object(13)
memory usage: 10.8+ MB


In [36]:
indv_stats = ids_w_ply_stats.loc[~((ids_w_ply_stats['recent_team'].isna()) & (ids_w_ply_stats['games'].isna())) | (ids_w_ply_stats['primary_position'] == 'DEF')]

In [37]:
indv_stats.columns

Index(['gsis_id', 'cleaned_name_yahoo', 'true_y_id', 'editorial_team_key',
       'editorial_team_abbr', 'player_id', 'player_key', 'primary_position',
       'season', 'last_name_yahoo',
       ...
       'pat_made', 'pat_att', 'pat_missed', 'pat_blocked', 'pat_pct',
       'gwfg_made', 'gwfg_att', 'gwfg_missed', 'gwfg_blocked',
       'gwfg_distance_list'],
      dtype='object', length=116)

In [38]:
ids[ids['gsis_id'] == '00-0023968']

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq


In [39]:
indv_stats = indv_stats.dropna(subset='cleaned_name_yahoo')
indv_stats.head()

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list
1,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,331.p.4269,K,2014.0,dawson,00-0004091_2014,4.0,125.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,31.0,5.0,1.0,55.0,0.806452,0.0,6.0,8.0,5.0,6.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948.0,250.0,45.0,33.0,33.0,0.0,0.0,1.000000,1.0,1.0,0.0,0.0,35
2,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,348.p.4269,K,2015.0,dawson,00-0004091_2015,9.0,102.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,27.0,1.0,2.0,54.0,0.888889,0.0,9.0,6.0,6.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30;25;47;33;43;22;53;31;26;42;35;54;26;44;27;2...,45,28;41,848.0,45.0,69.0,20.0,21.0,0.0,1.0,0.952381,1.0,1.0,0.0,0.0,23
3,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,359.p.4269,K,2016.0,dawson,00-0004091_2016,9.0,95.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,21.0,3.0,0.0,53.0,0.857143,0.0,4.0,6.0,7.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,29;39;22;36;33;48;47;28;49;48;46;49;53;33;36;3...,53;44;48,NaN,702.0,145.0,0.0,33.0,34.0,1.0,0.0,0.970588,0.0,0.0,0.0,0.0,NaN
4,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,371.p.4269,K,2017.0,dawson,00-0004091_2017,8.0,137.0,ARI,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,40.0,6.0,2.0,57.0,0.800000,1.0,7.0,10.0,10.0,4.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,24;27;40;30;37;29;43;50;32;28;25;32;43

In [40]:
indv_stats[indv_stats['primary_position'] == 'DEF']

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list
11805,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,331.p.100022,DEF,2014.0,cardinals,Arizona Cardinals_2014,4.0,213.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11806,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,348.p.100022,DEF,2015.0,cardinals,Arizona Cardinals_2015,9.0,253.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11807,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,359.p.100022,DEF,2016.0,cardinals,Arizona Cardinals_2016,9.0,235.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11808,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,371.p.100022,DEF,2017.0,cardinals,Arizona Cardinals_2017,8.0,216.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11809,Arizona Cardinals,arizona cardinals,100022.0

In [41]:
indv_stats[(indv_stats['def_tackles_for_loss'].notna()) & (indv_stats['def_tackles_for_loss'] > 0)]

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list
1882,00-0029051,jarrett boykin,26083.0,nfl.t.9,GB,26083.0,331.p.26083,WR,2014.0,boykin,00-0029051_2014,9.0,5.30,GB,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,3.0,12.0,23.0,0.0,0.0,0.0,133.0,27.0,1.0,-9.838036,0.0,0.172932,0.022599,0.030751,0.055424,0.0,2.0,0.0,1.0,1.0,12.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN
1975,00-0029239,brandon bolden,26389.0,nfl.t.13,LV,26389.0,348.p.26389,RB,2015.0,bolden,00-0029239_2015,6.0,69.70,NE,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,63.0,207.0,0.0,0.0,0.0,9.0,-16.403109,0.0,19.0,30.0,180.0,2.0,0.0,0.0,65.0,129.0,8.0,2.242350,0.0,2.769231,0.048077,0.013171,0.081335,0.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN
4375,00-0031697,jordan taylor,28727.0,nfl.t.7,Den,28727.0,371.p.28727,WR,2017.0,taylor,00-0031697_2017,5.0,27.20,DEN,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,13.0,20.0,142.0,0.0,0.0,0.0,206.0,45.0,5.0,-0.047186,0.0,0.689320,0.036298,0.043378,0.084811,0.0,4.0,0.0,0.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,11.0,103.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN
6031,00-0033251,zach pascal,30502.0,nfl.t.19,NYG,30502.0,390.p.30502,WR,2019.0,pascal,00-0033251_2019,11.0,137.20,IND,16.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,-0.704225,-42.273840,0.0,0.000000,2.0,16.0,0.0,0.0,0.0,1.0,0.907318,0.0,41.0,72.0,607.0,5.0,0.0,0.0,810.0,239.0,29.0,20.319164,1.0,0.749383,0.147541,0.196364,0.358766,0.0,6.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,20.0,0.0,0.0,10.0,263.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,

In [42]:
indv_stats = indv_stats.drop(columns=(indv_stats.columns[indv_stats.columns.str.startswith('def_')].tolist()))

In [43]:
indv_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7720 entries, 1 to 12190
Columns: 101 entries, gsis_id to gwfg_distance_list
dtypes: float64(88), object(13)
memory usage: 6.0+ MB


In [44]:
indv_stats = indv_stats.drop(columns=(indv_stats.columns[indv_stats.columns.str.startswith('fumble_recovery')].tolist()))
indv_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7720 entries, 1 to 12190
Data columns (total 96 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gsis_id                      7720 non-null   object 
 1   cleaned_name_yahoo           7720 non-null   object 
 2   true_y_id                    7720 non-null   float64
 3   editorial_team_key           7720 non-null   object 
 4   editorial_team_abbr          7720 non-null   object 
 5   player_id                    7720 non-null   float64
 6   player_key                   7720 non-null   object 
 7   primary_position             7720 non-null   object 
 8   season                       7720 non-null   float64
 9   last_name_yahoo              7720 non-null   object 
 10  gs_unq                       7720 non-null   object 
 11  bye_weeks.week               7720 non-null   float64
 12  player_points.total          7720 non-null   float64
 13  recent_team           

In [45]:
indv_stats = indv_stats.drop(columns=(indv_stats.columns[indv_stats.columns.str.startswith('gwfg')].tolist()))
indv_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7720 entries, 1 to 12190
Data columns (total 91 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gsis_id                      7720 non-null   object 
 1   cleaned_name_yahoo           7720 non-null   object 
 2   true_y_id                    7720 non-null   float64
 3   editorial_team_key           7720 non-null   object 
 4   editorial_team_abbr          7720 non-null   object 
 5   player_id                    7720 non-null   float64
 6   player_key                   7720 non-null   object 
 7   primary_position             7720 non-null   object 
 8   season                       7720 non-null   float64
 9   last_name_yahoo              7720 non-null   object 
 10  gs_unq                       7720 non-null   object 
 11  bye_weeks.week               7720 non-null   float64
 12  player_points.total          7720 non-null   float64
 13  recent_team           

In [46]:
indv_stats = indv_stats.drop(columns=(indv_stats.columns[indv_stats.columns.str.startswith('kickoff_return')].tolist()))
indv_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7720 entries, 1 to 12190
Data columns (total 89 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gsis_id                      7720 non-null   object 
 1   cleaned_name_yahoo           7720 non-null   object 
 2   true_y_id                    7720 non-null   float64
 3   editorial_team_key           7720 non-null   object 
 4   editorial_team_abbr          7720 non-null   object 
 5   player_id                    7720 non-null   float64
 6   player_key                   7720 non-null   object 
 7   primary_position             7720 non-null   object 
 8   season                       7720 non-null   float64
 9   last_name_yahoo              7720 non-null   object 
 10  gs_unq                       7720 non-null   object 
 11  bye_weeks.week               7720 non-null   float64
 12  player_points.total          7720 non-null   float64
 13  recent_team           

In [47]:
indv_stats = indv_stats.drop(columns=(indv_stats.columns[indv_stats.columns.str.startswith('punt_return')].tolist()))
indv_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7720 entries, 1 to 12190
Data columns (total 87 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gsis_id                      7720 non-null   object 
 1   cleaned_name_yahoo           7720 non-null   object 
 2   true_y_id                    7720 non-null   float64
 3   editorial_team_key           7720 non-null   object 
 4   editorial_team_abbr          7720 non-null   object 
 5   player_id                    7720 non-null   float64
 6   player_key                   7720 non-null   object 
 7   primary_position             7720 non-null   object 
 8   season                       7720 non-null   float64
 9   last_name_yahoo              7720 non-null   object 
 10  gs_unq                       7720 non-null   object 
 11  bye_weeks.week               7720 non-null   float64
 12  player_points.total          7720 non-null   float64
 13  recent_team           

In [48]:
indv_stats = indv_stats.drop(columns=(indv_stats.columns[indv_stats.columns.str.startswith('special_teams')].tolist()))
indv_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7720 entries, 1 to 12190
Data columns (total 86 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gsis_id                      7720 non-null   object 
 1   cleaned_name_yahoo           7720 non-null   object 
 2   true_y_id                    7720 non-null   float64
 3   editorial_team_key           7720 non-null   object 
 4   editorial_team_abbr          7720 non-null   object 
 5   player_id                    7720 non-null   float64
 6   player_key                   7720 non-null   object 
 7   primary_position             7720 non-null   object 
 8   season                       7720 non-null   float64
 9   last_name_yahoo              7720 non-null   object 
 10  gs_unq                       7720 non-null   object 
 11  bye_weeks.week               7720 non-null   float64
 12  player_points.total          7720 non-null   float64
 13  recent_team           

In [49]:
indv_stats = indv_stats.fillna(0)

In [50]:
indv_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7720 entries, 1 to 12190
Data columns (total 86 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gsis_id                      7720 non-null   object 
 1   cleaned_name_yahoo           7720 non-null   object 
 2   true_y_id                    7720 non-null   float64
 3   editorial_team_key           7720 non-null   object 
 4   editorial_team_abbr          7720 non-null   object 
 5   player_id                    7720 non-null   float64
 6   player_key                   7720 non-null   object 
 7   primary_position             7720 non-null   object 
 8   season                       7720 non-null   float64
 9   last_name_yahoo              7720 non-null   object 
 10  gs_unq                       7720 non-null   object 
 11  bye_weeks.week               7720 non-null   float64
 12  player_points.total          7720 non-null   float64
 13  recent_team           

In [105]:
team_stats = pd.read_csv('Raw/stats_def_nfl_raw_14to25.csv')
team_stats.head()

,Team,Tot Yds & TO.PA,Tot Yds & TO.Yds,Tot Yds & TO.Ply,Tot Yds & TO.Y/P,Tot Yds & TO.TO,Tot Yds & TO.FR,Tot Yds & TO.1stD,Passing.Cmp,Passing.Att,Passing.Yds,Passing.TD,Passing.Int,Passing.NY/A,Passing.1stD,Rushing.Att,Rushing.Yds,Rushing.TD,Rushing.Y/A,Rushing.1stD,Penalties.Pen,Penalties.Yds,Penalties.1stPy,Tot Yds & TO.Sc%,Tot Yds & TO.TO%,Tot Yds & TO.EXP,season
0,Seattle Seahawks,254.0,4274.0,924.0,4.6,24.0,11.0,277.0,313.0,507.0,2970.0,17.0,13.0,5.5,175.0,380.0,1304.0,8.0,3.4,70.0,70.0,613.0,32.0,26.9,12.3,40.92,2014
1,Kansas City Chiefs,281.0,5288.0,1024.0,5.2,14.0,8.0,310.0,318.0,545.0,3252.0,22.0,6.0,5.5,184.0,433.0,2036.0,4.0,4.7,108.0,107.0,851.0,18.0,32.6,6.7,-16.38,2014
2,Detroit Lions,282.0,4815.0,984.0,4.9,27.0,7.0,310.0,383.0,592.0,3706.0,23.0,20.0,5.8,209.0,350.0,1109.0,8.0,3.2,59.0,99.0,849.0,42.0,26.9,14.3,53.86,2014
3,Buffalo Bills,289.0,4995.0,1027.0,4.9,30.0,11.0,302.0,337.0,559.0,3292.0,16.0,19.0,5.4,171.0,414.0,1703.0,10.0,4.1,89.0,104.0,849.0,42.0,28.3,14.1,43.28,2014
4,Arizona Cardinals,299.0,5891.0,1010.0,5.8,25.0,7.0,300.0,365.0,579.0,4152.0,22.0,18.0,6.8,195.0,396.0,1739.0,9.0,4.4,77.0,130.0,1192.0,28.0,29.8,12.8,-9.75,2014


In [106]:
team_stats[team_stats['season'] == 2024].head()

,Team,Tot Yds & TO.PA,Tot Yds & TO.Yds,Tot Yds & TO.Ply,Tot Yds & TO.Y/P,Tot Yds & TO.TO,Tot Yds & TO.FR,Tot Yds & TO.1stD,Passing.Cmp,Passing.Att,Passing.Yds,Passing.TD,Passing.Int,Passing.NY/A,Passing.1stD,Rushing.Att,Rushing.Yds,Rushing.TD,Rushing.Y/A,Rushing.1stD,Penalties.Pen,Penalties.Yds,Penalties.1stPy,Tot Yds & TO.Sc%,Tot Yds & TO.TO%,Tot Yds & TO.EXP,season
350,Los Angeles Chargers,301.0,5514.0,1037.0,5.3,21.0,6.0,311.0,367.0,565.0,3517.0,24.0,15.0,5.8,170.0,426.0,1997.0,7.0,4.7,108.0,101.0,869.0,33.0,32.6,11.6,-12.31,2024
351,Philadelphia Eagles,303.0,4732.0,999.0,4.7,26.0,13.0,293.0,337.0,542.0,2961.0,22.0,13.0,5.1,160.0,416.0,1771.0,9.0,4.3,108.0,85.0,694.0,25.0,31.6,14.1,76.38,2024
352,Denver Broncos,311.0,5391.0,1093.0,4.9,25.0,10.0,327.0,393.0,606.0,3752.0,22.0,15.0,5.6,203.0,424.0,1639.0,10.0,3.9,82.0,101.0,723.0,42.0,31.6,12.4,34.83,2024
353,Kansas City Chiefs,326.0,5451.0,1026.0,5.3,20.0,7.0,328.0,375.0,569.0,3720.0,24.0,13.0,6.1,191.0,418.0,1731.0,13.0,4.1,112.0,107.0,849.0,25.0,36.5,11.4,-74.11,2024
354,Minnesota Vikings,332.0,5702.0,1078.0,5.3,33.0,9.0,325.0,415.0,637.0,4114.0,24.0,24.0,6.0,218.0,392.0,1588.0,12.0,4.1,83.0,141.0,1185.0,24.0,31.6,16.6,35.95,2024


In [107]:
team_stats = team_stats.rename(columns={'Team': 'def_team', 'Tot Yds & TO.PA': 'def_points_allowed', 'Tot Yds & TO.Ply' : 'def_plays_against', 'Tot Yds & TO.Y/P': 'def_yds_per_play', 'Tot Yds & TO.TO' : 'def_takeaways', 'Passing.Cmp': 'def_cmp_against', 
                           'Passing.Att': 'def_pass_attempts_against', 'Passing.Yds': 'def_passing_allowed', 'Passing.TD' : 'def_passing_td_allowed', 'Passing.NY/A': 'def_passing_ny_attempt', 'Passing.1stD': 'def_passing_1st',
                           'Rushing.Att': 'def_rushing_attamepts_against', 'Rushing.Yds': 'def_rushing_allowed', 'Rushing.TD': 'def_rushing_td_allowed', 'Rushing.Y/A' : 'def_rushing_ny_attempt', 'Rushing.1stD': 'def_rushing_1st',
                           'Penalties.Pen': 'def_penalties', 'Penalties.Yds': 'def_penalty_yds', 'Penalties.1stPy': 'def_penalty_1st', 'Tot Yds & TO.Sc%': 'def_scoring_drives%', 'Tot Yds & TO.TO%': 'def_TO_drives%', 'Tot Yds & TO.EXP': 'def_exp_pts', 'season': 'def_season'})

In [108]:
team_stats = team_stats.loc[:, team_stats.columns.str.startswith('def')]

In [109]:
team_stats.head()

,def_team,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season
0,Seattle Seahawks,254.0,924.0,4.6,24.0,313.0,507.0,2970.0,17.0,5.5,175.0,380.0,1304.0,8.0,3.4,70.0,70.0,613.0,32.0,26.9,12.3,40.92,2014
1,Kansas City Chiefs,281.0,1024.0,5.2,14.0,318.0,545.0,3252.0,22.0,5.5,184.0,433.0,2036.0,4.0,4.7,108.0,107.0,851.0,18.0,32.6,6.7,-16.38,2014
2,Detroit Lions,282.0,984.0,4.9,27.0,383.0,592.0,3706.0,23.0,5.8,209.0,350.0,1109.0,8.0,3.2,59.0,99.0,849.0,42.0,26.9,14.3,53.86,2014
3,Buffalo Bills,289.0,1027.0,4.9,30.0,337.0,559.0,3292.0,16.0,5.4,171.0,414.0,1703.0,10.0,4.1,89.0,104.0,849.0,42.0,28.3,14.1,43.28,2014
4,Arizona Cardinals,299.0,1010.0,5.8,25.0,365.0,579.0,4152.0,22.0,6.8,195.0,396.0,1739.0,9.0,4.4,77.0,130.0,1192.0,28.0,29.8,12.8,-9.75,2014


In [110]:
team_stats.loc[:, team_stats.columns.str.contains('%')] = team_stats.loc[:, team_stats.columns.str.contains('%')] / 100
team_stats.head()

,def_team,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season
0,Seattle Seahawks,254.0,924.0,4.6,24.0,313.0,507.0,2970.0,17.0,5.5,175.0,380.0,1304.0,8.0,3.4,70.0,70.0,613.0,32.0,0.269,0.123,40.92,2014
1,Kansas City Chiefs,281.0,1024.0,5.2,14.0,318.0,545.0,3252.0,22.0,5.5,184.0,433.0,2036.0,4.0,4.7,108.0,107.0,851.0,18.0,0.326,0.067,-16.38,2014
2,Detroit Lions,282.0,984.0,4.9,27.0,383.0,592.0,3706.0,23.0,5.8,209.0,350.0,1109.0,8.0,3.2,59.0,99.0,849.0,42.0,0.269,0.143,53.86,2014
3,Buffalo Bills,289.0,1027.0,4.9,30.0,337.0,559.0,3292.0,16.0,5.4,171.0,414.0,1703.0,10.0,4.1,89.0,104.0,849.0,42.0,0.283,0.141,43.28,2014
4,Arizona Cardinals,299.0,1010.0,5.8,25.0,365.0,579.0,4152.0,22.0,6.8,195.0,396.0,1739.0,9.0,4.4,77.0,130.0,1192.0,28.0,0.298,0.128,-9.75,2014


In [111]:
team_stats['def_team'] = team_stats['def_team'].str.lower()
team_stats.head()

,def_team,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season
0,seattle seahawks,254.0,924.0,4.6,24.0,313.0,507.0,2970.0,17.0,5.5,175.0,380.0,1304.0,8.0,3.4,70.0,70.0,613.0,32.0,0.269,0.123,40.92,2014
1,kansas city chiefs,281.0,1024.0,5.2,14.0,318.0,545.0,3252.0,22.0,5.5,184.0,433.0,2036.0,4.0,4.7,108.0,107.0,851.0,18.0,0.326,0.067,-16.38,2014
2,detroit lions,282.0,984.0,4.9,27.0,383.0,592.0,3706.0,23.0,5.8,209.0,350.0,1109.0,8.0,3.2,59.0,99.0,849.0,42.0,0.269,0.143,53.86,2014
3,buffalo bills,289.0,1027.0,4.9,30.0,337.0,559.0,3292.0,16.0,5.4,171.0,414.0,1703.0,10.0,4.1,89.0,104.0,849.0,42.0,0.283,0.141,43.28,2014
4,arizona cardinals,299.0,1010.0,5.8,25.0,365.0,579.0,4152.0,22.0,6.8,195.0,396.0,1739.0,9.0,4.4,77.0,130.0,1192.0,28.0,0.298,0.128,-9.75,2014


In [112]:
team_stats['def_team'].value_counts().sort_index()

def_team
arizona cardinals           12
atlanta falcons             12
avg team                    12
avg tm/g                    12
baltimore ravens            12
buffalo bills               12
carolina panthers           12
chicago bears               12
cincinnati bengals          12
cleveland browns            12
dallas cowboys              12
denver broncos              12
detroit lions               12
green bay packers           12
houston texans              12
indianapolis colts          12
jacksonville jaguars        12
kansas city chiefs          12
las vegas raiders            6
league total                12
los angeles chargers         9
los angeles rams            10
miami dolphins              12
minnesota vikings           12
new england patriots        12
new orleans saints          12
new york giants             12
new york jets               12
oakland raiders              6
philadelphia eagles         12
pittsburgh steelers         12
san diego chargers           3

In [113]:
team_stats['def_team'] = team_stats['def_team'].replace({'oakland raiders': 'las vegas raiders', 'san diego chargers': 'los angeles chargers', 'st. louis rams' : 'los angeles rams', 'washington football team': 'washington commanders', 'washington redskins': 'washington commanders'})
team_stats['def_team'].value_counts().sort_index()

def_team
arizona cardinals        12
atlanta falcons          12
avg team                 12
avg tm/g                 12
baltimore ravens         12
buffalo bills            12
carolina panthers        12
chicago bears            12
cincinnati bengals       12
cleveland browns         12
dallas cowboys           12
denver broncos           12
detroit lions            12
green bay packers        12
houston texans           12
indianapolis colts       12
jacksonville jaguars     12
kansas city chiefs       12
las vegas raiders        12
league total             12
los angeles chargers     12
los angeles rams         12
miami dolphins           12
minnesota vikings        12
new england patriots     12
new orleans saints       12
new york giants          12
new york jets            12
philadelphia eagles      12
pittsburgh steelers      12
san francisco 49ers      12
seattle seahawks         12
tampa bay buccaneers     12
tennessee titans         12
washington commanders    12
Name: count

In [114]:
team_stats['merge_id'] = team_stats['def_team'] + '_' + team_stats['def_season'].astype(str)
team_stats.head()

,def_team,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,merge_id
0,seattle seahawks,254.0,924.0,4.6,24.0,313.0,507.0,2970.0,17.0,5.5,175.0,380.0,1304.0,8.0,3.4,70.0,70.0,613.0,32.0,0.269,0.123,40.92,2014,seattle seahawks_2014
1,kansas city chiefs,281.0,1024.0,5.2,14.0,318.0,545.0,3252.0,22.0,5.5,184.0,433.0,2036.0,4.0,4.7,108.0,107.0,851.0,18.0,0.326,0.067,-16.38,2014,kansas city chiefs_2014
2,detroit lions,282.0,984.0,4.9,27.0,383.0,592.0,3706.0,23.0,5.8,209.0,350.0,1109.0,8.0,3.2,59.0,99.0,849.0,42.0,0.269,0.143,53.86,2014,detroit lions_2014
3,buffalo bills,289.0,1027.0,4.9,30.0,337.0,559.0,3292.0,16.0,5.4,171.0,414.0,1703.0,10.0,4.1,89.0,104.0,849.0,42.0,0.283,0.141,43.28,2014,buffalo bills_2014
4,arizona cardinals,299.0,1010.0,5.8,25.0,365.0,579.0,4152.0,22.0,6.8,195.0,396.0,1739.0,9.0,4.4,77.0,130.0,1192.0,28.0,0.298,0.128,-9.75,2014,arizona cardinals_2014


In [120]:
conv = pd.read_csv('Raw/conversions_raw_14to25.csv')
conv.head()

,Rk,Tm,G,3DAtt,3DConv,3D%,4DAtt,4DConv,4D%,RZAtt,RZTD,RZPct,season
0,1.0,Indianapolis Colts,16.0,201.0,67.0,33.3%,17.0,4.0,23.5%,50.0,33.0,66.0%,2014
1,2.0,Pittsburgh Steelers,16.0,189.0,71.0,37.6%,14.0,10.0,71.4%,50.0,27.0,54.0%,2014
2,3.0,New York Giants,16.0,200.0,72.0,36.0%,10.0,4.0,40.0%,51.0,25.0,49.0%,2014
3,4.0,Buffalo Bills,16.0,220.0,73.0,33.2%,11.0,5.0,45.5%,44.0,22.0,50.0%,2014
4,5.0,Houston Texans,16.0,218.0,74.0,33.9%,17.0,6.0,35.3%,46.0,22.0,47.8%,2014


In [121]:
conv = conv.drop(columns='Rk')


In [122]:
conv['Tm'] = conv['Tm'].str.lower()
conv.head()

,Tm,G,3DAtt,3DConv,3D%,4DAtt,4DConv,4D%,RZAtt,RZTD,RZPct,season
0,indianapolis colts,16.0,201.0,67.0,33.3%,17.0,4.0,23.5%,50.0,33.0,66.0%,2014
1,pittsburgh steelers,16.0,189.0,71.0,37.6%,14.0,10.0,71.4%,50.0,27.0,54.0%,2014
2,new york giants,16.0,200.0,72.0,36.0%,10.0,4.0,40.0%,51.0,25.0,49.0%,2014
3,buffalo bills,16.0,220.0,73.0,33.2%,11.0,5.0,45.5%,44.0,22.0,50.0%,2014
4,houston texans,16.0,218.0,74.0,33.9%,17.0,6.0,35.3%,46.0,22.0,47.8%,2014


In [123]:
conv = conv.rename(columns={'Tm' : 'def_team','3DAtt': 'def_3Datt', '3D%': 'def_3D%', '4DAtt': 'def_4Datt', '4D%': 'def_4D%', 'RZAtt': 'def_rz_att', 'RZPct': 'def_rz%', 'season': 'def_season'})

In [124]:
conv = conv.loc[:, conv.columns.str.startswith('def')]
conv.head()

,def_team,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%,def_season
0,indianapolis colts,201.0,33.3%,17.0,23.5%,50.0,66.0%,2014
1,pittsburgh steelers,189.0,37.6%,14.0,71.4%,50.0,54.0%,2014
2,new york giants,200.0,36.0%,10.0,40.0%,51.0,49.0%,2014
3,buffalo bills,220.0,33.2%,11.0,45.5%,44.0,50.0%,2014
4,houston texans,218.0,33.9%,17.0,35.3%,46.0,47.8%,2014


In [132]:
conv[conv.columns[conv.columns.str.contains('%')]] = conv[conv.columns[conv.columns.str.contains('%')]].apply(lambda x: x.str.strip('%').astype(float) / 100)
conv.head()

,def_team,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%,def_season
0,indianapolis colts,201.0,0.333,17.0,0.235,50.0,0.660,2014
1,pittsburgh steelers,189.0,0.376,14.0,0.714,50.0,0.540,2014
2,new york giants,200.0,0.360,10.0,0.400,51.0,0.490,2014
3,buffalo bills,220.0,0.332,11.0,0.455,44.0,0.500,2014
4,houston texans,218.0,0.339,17.0,0.353,46.0,0.478,2014


In [133]:
conv['def_team'] = conv['def_team'].replace({'oakland raiders': 'las vegas raiders', 'san diego chargers': 'los angeles chargers', 'st. louis rams' : 'los angeles rams', 'washington football team': 'washington commanders', 'washington redskins': 'washington commanders'})

In [134]:
conv['def_team'].value_counts()

def_team
indianapolis colts       12
pittsburgh steelers      12
new york giants          12
buffalo bills            12
houston texans           12
seattle seahawks         12
detroit lions            12
cincinnati bengals       12
kansas city chiefs       12
arizona cardinals        12
denver broncos           12
los angeles rams         12
green bay packers        12
new england patriots     12
las vegas raiders        12
chicago bears            12
san francisco 49ers      12
philadelphia eagles      12
los angeles chargers     12
washington commanders    12
baltimore ravens         12
dallas cowboys           12
carolina panthers        12
minnesota vikings        12
cleveland browns         12
tampa bay buccaneers     12
tennessee titans         12
atlanta falcons          12
miami dolphins           12
new york jets            12
new orleans saints       12
jacksonville jaguars     12
league total             12
Name: count, dtype: int64

In [135]:
conv['merge_id'] = conv['def_team'] + '_' + conv['def_season'].astype(str)
conv.head()

,def_team,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%,def_season,merge_id
0,indianapolis colts,201.0,0.333,17.0,0.235,50.0,0.660,2014,indianapolis colts_2014
1,pittsburgh steelers,189.0,0.376,14.0,0.714,50.0,0.540,2014,pittsburgh steelers_2014
2,new york giants,200.0,0.360,10.0,0.400,51.0,0.490,2014,new york giants_2014
3,buffalo bills,220.0,0.332,11.0,0.455,44.0,0.500,2014,buffalo bills_2014
4,houston texans,218.0,0.339,17.0,0.353,46.0,0.478,2014,houston texans_2014


In [136]:
conv = conv.drop(columns=['def_team', 'def_season'])
conv.head()

,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%,merge_id
0,201.0,0.333,17.0,0.235,50.0,0.660,indianapolis colts_2014
1,189.0,0.376,14.0,0.714,50.0,0.540,pittsburgh steelers_2014
2,200.0,0.360,10.0,0.400,51.0,0.490,new york giants_2014
3,220.0,0.332,11.0,0.455,44.0,0.500,buffalo bills_2014
4,218.0,0.339,17.0,0.353,46.0,0.478,houston texans_2014


In [137]:
indv_stats['gs_unq'] = indv_stats['gs_unq'].str.lower()
indv_stats[indv_stats['primary_position'] == 'DEF']

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct
11805,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,331.p.100022,DEF,2014.0,cardinals,arizona cardinals_2014,4.0,213.88,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11806,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,348.p.100022,DEF,2015.0,cardinals,arizona cardinals_2015,9.0,253.68,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11807,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,359.p.100022,DEF,2016.0,cardinals,arizona cardinals_2016,9.0,235.98,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11808,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,371.p.100022,DEF,2017.0,cardinals,arizona cardinals_2017,8.0,216.85,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11809,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,380.p.100022,DEF,2018.0,cardinals,arizona cardinals_2018,9.0,181.37,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12186,Washington Commanders,washington commanders,100028.0,nfl.t.28,Was,100028.0,406.p.100028,DEF,2021.0,commanders,washington commanders_2021,9.0,171.76,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [138]:
def_stats = indv_stats.merge(team_stats, how='left', left_on='gs_unq', right_on='merge_id')
def_stats[def_stats['primary_position'] == 'DEF']

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_team,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,merge_id
7336,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,331.p.100022,DEF,2014.0,cardinals,arizona cardinals_2014,4.0,213.88,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,arizona cardinals,299.0,1010.0,5.8,25.0,365.0,579.0,4152.0,22.0,6.8,195.0,396.0,1739.0,9.0,4.4,77.0,130.0,1192.0,28.0,0.298,0.128,-9.75,2014.0,arizona cardinals_2014
7337,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,348.p.100022,DEF,2015.0,cardinals,arizona cardinals_2015,9.0,253.68,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,arizona cardinals,313.0,982.0,5.2,33.0,343.0,573.0,3687.0,24.0,6.1,177.0,373.0,1460.0,9.0,3.9,79.0,130.0,1202.0,28.0,0.317,0.167,34.93,2015.0,arizona cardinals_2015
7338,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,359.p.100022,DEF,2016.0,cardinals,arizona cardinals_2016,9.0,235.98,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,arizona cardinals,362.0,1019.0,4.8,28.0,344.0,547.0,3365.0,21.0,5.7,175.0,424.0,1518.0,16.0,3.6,90.0,123.0,1010.0,28.0,0.325,0.129,81.68,2016.0,arizona cardinals_2016
7339,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,371.p.100022,DEF,2017.0,cardinals,arizona cardinals_2017,8.0,216.85,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,arizona cardinals,361.0,1008.0,4.9,21.0,343.0,564.0,3541.0,24.0,5.9,187.0,407.0,1432.0,12.0,3.5,68.0,129.0,1128.0,28.0,0.315,0.105,143.08,2017.0,arizona cardin

In [139]:
def_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7720 entries, 0 to 7719
Columns: 110 entries, gsis_id to merge_id
dtypes: float64(96), object(14)
memory usage: 6.5+ MB


In [143]:
conv_stats = def_stats.merge(conv, how='left', left_on='gs_unq', right_on = 'merge_id')
conv_stats[conv_stats['primary_position'] == 'DEF']

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_team,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,merge_id_x,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%,merge_id_y
7336,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,331.p.100022,DEF,2014.0,cardinals,arizona cardinals_2014,4.0,213.88,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,arizona cardinals,299.0,1010.0,5.8,25.0,365.0,579.0,4152.0,22.0,6.8,195.0,396.0,1739.0,9.0,4.4,77.0,130.0,1192.0,28.0,0.298,0.128,-9.75,2014.0,arizona cardinals_2014,215.0,0.377,13.0,0.385,41.0,0.439,arizona cardinals_2014
7337,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,348.p.100022,DEF,2015.0,cardinals,arizona cardinals_2015,9.0,253.68,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,arizona cardinals,313.0,982.0,5.2,33.0,343.0,573.0,3687.0,24.0,6.1,177.0,373.0,1460.0,9.0,3.9,79.0,130.0,1202.0,28.0,0.317,0.167,34.93,2015.0,arizona cardinals_2015,210.0,0.357,17.0,0.471,47.0,0.553,arizona cardinals_2015
7338,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,359.p.100022,DEF,2016.0,cardinals,arizona cardinals_2016,9.0,235.98,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,arizona cardinals,362.0,1019.0,4.8,28.0,344.0,547.0,3365.0,21.0,5.7,175.0,424.0,1518.0,16.0,3.6,90.0,123.0,1010.0,28.0,0.325,0.129,81.68,2016.0,arizona cardinals_2016,219.0,0.388,15.0,0.467,45.0,0.556,arizona cardinals_2016
7339,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,371.p.100022,DEF,2017.0,cardinals,arizona cardinals_2017,8.0,216.85,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [144]:
conv_stats = conv_stats.drop(columns=['def_team', 'merge_id_x', 'merge_id_y'])
conv_stats.head()

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%
0,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,331.p.4269,K,2014.0,dawson,00-0004091_2014,4.0,125.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,31.0,5.0,1.0,55.0,0.806452,0.0,6.0,8.0,5.0,6.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948.0,250.0,45.0,33.0,33.0,0.0,0.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,348.p.4269,K,2015.0,dawson,00-0004091_2015,9.0,102.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,27.0,1.0,2.0,54.0,0.888889,0.0,9.0,6.0,6.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30;25;47;33;43;22;53;31;26;42;35;54;26;44;27;2...,45,28;41,848.0,45.0,69.0,20.0,21.0,0.0,1.0,0.952381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,359.p.4269,K,2016.0,dawson,00-0004091_2016,9.0,95.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,21.0,3.0,0.0,53.0,0.857143,0.0,4.0,6.0,7.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,29;39;22;36;33;48;47;28;49;48;46;49;53;33;36;3...,53;44;48,0,702.0,145.0,0.0,33.0,34.0,1.0,0.0,0.970588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,371.p.4269,K,2017.0,dawson,00-0004091_2017,8.0,137.0,ARI,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,40.0,6.0,2.0,57.0,0.800000,1.0,7.0,10.0,10.0,4.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,24;27;40;30;37;29;43;50;32;28;25;32;43;34;42;4...,32;42;36;32;38;40,51;45,1183.0,220.0,96.0,23.0,26.0,1.0,2.0,0.884615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [145]:
conv_stats = conv_stats.drop(columns=['editorial_team_key', 'editorial_team_abbr', 'true_y_id', 'player_id', 'last_name_yahoo'])
conv_stats.head()

,gsis_id,cleaned_name_yahoo,player_key,primary_position,season,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%
0,00-0004091,phil dawson,331.p.4269,K,2014.0,00-0004091_2014,4.0,125.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,31.0,5.0,1.0,55.0,0.806452,0.0,6.0,8.0,5.0,6.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948.0,250.0,45.0,33.0,33.0,0.0,0.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00-0004091,phil dawson,348.p.4269,K,2015.0,00-0004091_2015,9.0,102.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,27.0,1.0,2.0,54.0,0.888889,0.0,9.0,6.0,6.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30;25;47;33;43;22;53;31;26;42;35;54;26;44;27;2...,45,28;41,848.0,45.0,69.0,20.0,21.0,0.0,1.0,0.952381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00-0004091,phil dawson,359.p.4269,K,2016.0,00-0004091_2016,9.0,95.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,21.0,3.0,0.0,53.0,0.857143,0.0,4.0,6.0,7.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,29;39;22;36;33;48;47;28;49;48;46;49;53;33;36;3...,53;44;48,0,702.0,145.0,0.0,33.0,34.0,1.0,0.0,0.970588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00-0004091,phil dawson,371.p.4269,K,2017.0,00-0004091_2017,8.0,137.0,ARI,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,40.0,6.0,2.0,57.0,0.800000,1.0,7.0,10.0,10.0,4.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,24;27;40;30;37;29;43;50;32;28;25;32;43;34;42;4...,32;42;36;32;38;40,51;45,1183.0,220.0,96.0,23.0,26.0,1.0,2.0,0.884615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00-0004091,phil dawson,380.p.4269,K,2018.0,00-0004091_2018,9.0,31.0,ARI,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [147]:
conv_stats['season'].isna().sum()

np.int64(0)

In [148]:
conv_stats['season'] = conv_stats['season'].astype(int)
conv_stats.head()

,gsis_id,cleaned_name_yahoo,player_key,primary_position,season,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%
0,00-0004091,phil dawson,331.p.4269,K,2014,00-0004091_2014,4.0,125.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,31.0,5.0,1.0,55.0,0.806452,0.0,6.0,8.0,5.0,6.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948.0,250.0,45.0,33.0,33.0,0.0,0.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00-0004091,phil dawson,348.p.4269,K,2015,00-0004091_2015,9.0,102.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,27.0,1.0,2.0,54.0,0.888889,0.0,9.0,6.0,6.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30;25;47;33;43;22;53;31;26;42;35;54;26;44;27;2...,45,28;41,848.0,45.0,69.0,20.0,21.0,0.0,1.0,0.952381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00-0004091,phil dawson,359.p.4269,K,2016,00-0004091_2016,9.0,95.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,21.0,3.0,0.0,53.0,0.857143,0.0,4.0,6.0,7.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,29;39;22;36;33;48;47;28;49;48;46;49;53;33;36;3...,53;44;48,0,702.0,145.0,0.0,33.0,34.0,1.0,0.0,0.970588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00-0004091,phil dawson,371.p.4269,K,2017,00-0004091_2017,8.0,137.0,ARI,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,40.0,6.0,2.0,57.0,0.800000,1.0,7.0,10.0,10.0,4.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,24;27;40;30;37;29;43;50;32;28;25;32;43;34;42;4...,32;42;36;32;38;40,51;45,1183.0,220.0,96.0,23.0,26.0,1.0,2.0,0.884615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00-0004091,phil dawson,380.p.4269,K,2018,00-0004091_2018,9.0,31.0,ARI,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [152]:
conv_stats['recent_team'].value_counts()

recent_team
0      384
NYJ    247
CLE    242
TEN    242
NYG    241
JAX    240
ARI    239
NO     238
SF     235
HOU    234
WAS    233
CHI    232
DET    232
BAL    232
BUF    231
TB     231
IND    230
PHI    229
LAC    229
CAR    229
MIA    228
NE     228
LA     228
KC     225
DEN    223
LV     223
CIN    223
SEA    219
ATL    218
GB     218
PIT    218
DAL    217
MIN    202
Name: count, dtype: int64

In [153]:
conv_stats[conv_stats['primary_position'] == 'DEF']

,gsis_id,cleaned_name_yahoo,player_key,primary_position,season,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%
7336,Arizona Cardinals,arizona cardinals,331.p.100022,DEF,2014,arizona cardinals_2014,4.0,213.88,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,299.0,1010.0,5.8,25.0,365.0,579.0,4152.0,22.0,6.8,195.0,396.0,1739.0,9.0,4.4,77.0,130.0,1192.0,28.0,0.298,0.128,-9.75,2014.0,215.0,0.377,13.0,0.385,41.0,0.439
7337,Arizona Cardinals,arizona cardinals,348.p.100022,DEF,2015,arizona cardinals_2015,9.0,253.68,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,313.0,982.0,5.2,33.0,343.0,573.0,3687.0,24.0,6.1,177.0,373.0,1460.0,9.0,3.9,79.0,130.0,1202.0,28.0,0.317,0.167,34.93,2015.0,210.0,0.357,17.0,0.471,47.0,0.553
7338,Arizona Cardinals,arizona cardinals,359.p.100022,DEF,2016,arizona cardinals_2016,9.0,235.98,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,362.0,1019.0,4.8,28.0,344.0,547.0,3365.0,21.0,5.7,175.0,424.0,1518.0,16.0,3.6,90.0,123.0,1010.0,28.0,0.325,0.129,81.68,2016.0,219.0,0.388,15.0,0.467,45.0,0.556
7339,Arizona Cardinals,arizona cardinals,371.p.100022,DEF,2017,arizona cardinals_2017,8.0,216.85,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,361.0,1008.0,4.9,21.0,343.0,564.0,3541.0,24.0,5.9,187.0,407.0,1432.0,12.0,3.5,68.0,129.0,1128.0,28.0,0.315,0.105,143.08,2017.0,224.0,0.357,16.0,0.438,46.0,0.587
7340,Arizona Cardinals,arizona cardinals,380.p.100022,DEF,2018,arizona cardinals_2018,9.0,181.37,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [154]:
conv_stats.loc[conv_stats['recent_team'] == 0, 'recent_team'] = conv_stats.loc[conv_stats['recent_team'] == 0, 'cleaned_name_yahoo'] 
conv_stats[conv_stats['primary_position'] == 'DEF']

,gsis_id,cleaned_name_yahoo,player_key,primary_position,season,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%
7336,Arizona Cardinals,arizona cardinals,331.p.100022,DEF,2014,arizona cardinals_2014,4.0,213.88,arizona cardinals,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,299.0,1010.0,5.8,25.0,365.0,579.0,4152.0,22.0,6.8,195.0,396.0,1739.0,9.0,4.4,77.0,130.0,1192.0,28.0,0.298,0.128,-9.75,2014.0,215.0,0.377,13.0,0.385,41.0,0.439
7337,Arizona Cardinals,arizona cardinals,348.p.100022,DEF,2015,arizona cardinals_2015,9.0,253.68,arizona cardinals,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,313.0,982.0,5.2,33.0,343.0,573.0,3687.0,24.0,6.1,177.0,373.0,1460.0,9.0,3.9,79.0,130.0,1202.0,28.0,0.317,0.167,34.93,2015.0,210.0,0.357,17.0,0.471,47.0,0.553
7338,Arizona Cardinals,arizona cardinals,359.p.100022,DEF,2016,arizona cardinals_2016,9.0,235.98,arizona cardinals,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,362.0,1019.0,4.8,28.0,344.0,547.0,3365.0,21.0,5.7,175.0,424.0,1518.0,16.0,3.6,90.0,123.0,1010.0,28.0,0.325,0.129,81.68,2016.0,219.0,0.388,15.0,0.467,45.0,0.556
7339,Arizona Cardinals,arizona cardinals,371.p.100022,DEF,2017,arizona cardinals_2017,8.0,216.85,arizona cardinals,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,361.0,1008.0,4.9,21.0,343.0,564.0,3541.0,24.0,5.9,187.0,407.0,1432.0,12.0,3.5,68.0,129.0,1128.0,28.0,0.315,0.105,143.08,2017.0,224.0,0.357,16.0,0.438,46.0,0.587
7340,Arizona Cardinals,arizona cardinals,380.p.100022,DEF,2018,arizona cardinals_2018,9.0,181.37,arizona cardinals,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [155]:
conv_stats['recent_team'].value_counts()

recent_team
NYJ                      247
CLE                      242
TEN                      242
NYG                      241
JAX                      240
                        ... 
san francisco 49ers       12
seattle seahawks          12
tampa bay buccaneers      12
tennessee titans          12
washington commanders     12
Name: count, Length: 64, dtype: int64

In [161]:
nfl_teams = {
    "ARI": "arizona cardinals",
    "ATL": "atlanta falcons",
    "BAL": "baltimore ravens",
    "BUF": "buffalo bills",
    "CAR": "carolina panthers",
    "CHI": "chicago bears",
    "CIN": "cincinnati bengals",
    "CLE": "cleveland browns",
    "DAL": "dallas cowboys",
    "DEN": "denver broncos",
    "DET": "detroit lions",
    "GB": "green bay packers",
    "HOU": "houston texans",
    "IND": "indianapolis colts",
    "JAX": "jacksonville jaguars",
    "KC": "kansas city chiefs",
    "LAC": "los angeles chargers",
    "LAR": "los angeles rams",
    "LV": "las vegas raiders",
    "MIA": "miami dolphins",
    "MIN": "minnesota vikings",
    "NE": "new england patriots",
    "NO": "new orleans saints",
    "NYG": "new york giants",
    "NYJ": "new york jets",
    "PHI": "philadelphia eagles",
    "PIT": "pittsburgh steelers",
    "SF": "san francisco 49ers",
    "SEA": "seattle seahawks",
    "TB": "tampa bay buccaneers",
    "TEN": "tennessee titans",
    "WAS": "washington commanders"
}

In [163]:
conv_stats['team'] = conv_stats['recent_team'].map(nfl_teams).fillna(conv_stats['cleaned_name_yahoo'])

In [164]:
conv_stats.head()

,gsis_id,cleaned_name_yahoo,player_key,primary_position,season,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%,team
0,00-0004091,phil dawson,331.p.4269,K,2014,00-0004091_2014,4.0,125.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,31.0,5.0,1.0,55.0,0.806452,0.0,6.0,8.0,5.0,6.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948.0,250.0,45.0,33.0,33.0,0.0,0.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,san francisco 49ers
1,00-0004091,phil dawson,348.p.4269,K,2015,00-0004091_2015,9.0,102.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,27.0,1.0,2.0,54.0,0.888889,0.0,9.0,6.0,6.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30;25;47;33;43;22;53;31;26;42;35;54;26;44;27;2...,45,28;41,848.0,45.0,69.0,20.0,21.0,0.0,1.0,0.952381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,san francisco 49ers
2,00-0004091,phil dawson,359.p.4269,K,2016,00-0004091_2016,9.0,95.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,21.0,3.0,0.0,53.0,0.857143,0.0,4.0,6.0,7.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,29;39;22;36;33;48;47;28;49;48;46;49;53;33;36;3...,53;44;48,0,702.0,145.0,0.0,33.0,34.0,1.0,0.0,0.970588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,san francisco 49ers
3,00-0004091,phil dawson,371.p.4269,K,2017,00-0004091_2017,8.0,137.0,ARI,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,40.0,6.0,2.0,57.0,0.800000,1.0,7.0,10.0,10.0,4.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,24;27;40;30;37;29;43;50;32;28;25;32;43;34;42;4...,32;42;36;32;38;40,51;45,1183.0,220.0,96.0,23.0,26.0,1.0,2.0,0.884615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,arizona cardinals
4,00-0004091,phil dawson,380.p.4269,K,2018,00-0004091_2018,9.0,31.0,ARI,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [167]:
conv_stats.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7720 entries, 0 to 7719
Data columns (total 110 columns):
 #    Column                         Non-Null Count  Dtype  
---   ------                         --------------  -----  
 0    gsis_id                        7720 non-null   object 
 1    cleaned_name_yahoo             7720 non-null   object 
 2    player_key                     7720 non-null   object 
 3    primary_position               7720 non-null   object 
 4    season                         7720 non-null   int64  
 5    gs_unq                         7720 non-null   object 
 6    bye_weeks.week                 7720 non-null   float64
 7    player_points.total            7720 non-null   float64
 8    recent_team                    7720 non-null   object 
 9    games                          7720 non-null   float64
 10   completions                    7720 non-null   float64
 11   attempts                       7720 non-null   float64
 12   passing_yards                  7

NEXT STEP IS TO FILL THE NULLS IN TEH DEF COLUMNS